# A Basic Tutorial of Stable Diffusion with Math📐 and Code💻

As learners like you, when we first touch upon the state-of-art model of Stable Diffusion, we found it complicated and perplexing. Based of empirical results and combined with multiple distinct parts, the model itself is hard to appreciate and even hard to follow without a clear and logical path😖. If you feel the same way, don't worry, we were all at the same boat😤. As learners, we looked through different materials and tutorials, but we could not find something that both comprehensive enough logic-wise or coding-wise😔.

Therefore, we aim to create a natural and straightforward introduction in both the foundation and implementation of the model🤗.

So let's begin our journey!

# What is Stable Diffusion?

According to Wikipedia and its github page, Stable Diffusion is a deep learning, text-to-image model released in 2022 based on diffusion techniques. From this, we know its backbone is essentially a model called diffusion. So, before we dive into the actual stable diffusion, we think it's better to learn about what is Diffusion. Or more specifically, DDPM - Denoising Diffusion Probabilistic Models. 

# Denoising Diffusion Probabilistic Models

If you are intereted in reading the original paper of DDPM by Ho et.al published 2020, here is the link for the paper: https://arxiv.org/pdf/2006.11239.pdf.

However, the original paper is really math-heavy, and skipped a lot of details. Thus, we provide our understanding and visions which should be slighly easier (?) to comprehend.

In short, the ddpm (diffusion) model is basically a process of adding noise and denoising, which could be easliy divided into 2 seperate directions, the forward process - adding noise and the reverse process - denosing. The general idea behind this model is, if by defined, we know how to iteratively add noise to a image that let it eventually become a completely noisy image (in a sense equals to random samples), could we use some deep learning networks to learn the noise added to the model in each step?

And if we could accomplish that, does it mean that we have a way to predict the added noise for each state during the process (the difference of noise level between each timestep)? Then using the noise we predicted, does it mean that we could start from a random noise, and continuing substract the noise we predicted, which at the end give us a image without noise? That is the generative result we want. 

Above is a very brief introduciton/ intuition for you to have something in mind before we head into the detail maths. But maths is something that we not able to avoid. I'll start with the forward process.

# Forward Process - Adding Noise

![](forward_pass.png)

- In the forward process, the paper defined $X_t = \sqrt{a_t}X_{t-1} + \sqrt{1-a_t}Z_t$, 
    - where $X_t$ is the image at timestep $t$, after added noise t times. 
    - $Z_t$ is the noise added from Normal Distribution ~ N(0, 1)
    - Yes, we need to also explain what timestep $t$ means. It is the number of steps we take to reach the final noisy image, shown in the figure above, timestep is the underscore number below each state of adding noise to the previous image.  
    - $a_t$ is the noise level at timestep $t$, and it is defined by using a function $a_t = 1 - \beta_t$ where $\beta_t$ is a hyperparameter that increases through timesteps, which t increases $\beta_t$ also increase. This would cause $a_t$ to decrease through timesteps.
        - So why? Why we defined $a_t$ in this way? Observe the formula, when $a_t$ decreases through timestep, $\sqrt{1-a_t}$ would conversely increase. This means that the noise added to the image would increase through timesteps. What it means? Imagine we start from a clean image, and we add noise to it, even a little bit of noise would be noticeable. But if we add noise to a noisy image, the noise added would be insignificant. As a result, with more timesteps, we also need to add more noise to make it be more inflential. This is the logic behind the definition of $a_t$.

- However, we don't want to add noise actually timestep by timestep, we want to add noise to the image in one go. If we have our image $X_0$, we want to have $X_T$ immediately. So now we want to relate $X_T$ to $X_0$ directly. 
    - We can do this by expanding the formula $X_t = \sqrt{a_t}X_{t-1} + \sqrt{1-a_t}Z_t$. 
    - => $X_t = \sqrt{a_t}(\sqrt{a_{t-1}}X_{t-2} + \sqrt{1-a_{t-1}}Z_{t-1}) + \sqrt{1-a_t}Z_t$
    - => $X_t = \sqrt{a_ta_{t-1}}X_{t-2} + \sqrt{a_t(1-a_{t-1})}Z_{t-1} + \sqrt{1-a_t}Z_t$
    - For Gaussian distribution, if we multiply two Gaussian distribution, in this case $Z_t$ and $Z_{t-1}$ 
        - if we multiply a Gaussian distribution with constant $c$: new $\mu' = c\mu$, new $\sigma' = |c|\sigma => \sigma'^2 = |c|\sigma^2$
        - $\sqrt{a_t(1-a_{t-1})}Z_{t-1} \sim N(0, a_t(1-a_{t-1}))$ and $\sqrt{1-a_t}Z_t \sim N(0, 1-a_t)$ are still Gaussian distributions
        - for any Guassion distribution $N(\mu_1, \sigma_1^2)$ and $N(\mu_2, \sigma_2^2)$, if they are independent, then $N(\mu_1, \sigma_1^2) + N(\mu_2, \sigma_2^2) \sim N(\mu_1 + \mu_2, \sigma_1^2 + \sigma_2^2)$
    - => $X_t = \sqrt{a_ta_{t-1}}X_{t-2} + \sqrt{1 - a_t + a_t -a_ta_{t-1}}\bar Z$ where $\bar Z \sim N(0, 1)$
    - => $X_t = \sqrt{a_ta_{t-1}}X_{t-2} + \sqrt{1 - a_ta_{t-1}}\bar Z$
    - continue this process, we can get $X_T = \sqrt{a_ta_{t-1}...a_1}X_0 + \sqrt{1 - a_ta_{t-1}...a_1}\bar Z$
- So if let $\bar a_t = \prod_1^t a_i$, 
- => $X_t = \sqrt{\bar a_t}X_0 + \sqrt{1 - \bar a_t}\bar Z$

And this is our final equation for adding noise in the forward process. It's possible you find the math above is a bit hard to follow, you should remember this formula, and we will use it in actual implementation.
This concludes the forward process. Hurrah! We are halfway there! 😀 

Next, we will move on to the reverse process - denoising.

# Reverse Process - Denoising

![](backward_pass.png)

As the name suggests, the reverse process is the opposite of the forward process. In the forward process, we add noise to the image, and in the reverse process, we remove the noise from the image. So if we are given the noisy image $X_t$, how can we know the less noisy image $X_{t-1}$?

Basically, we want the probability $q(X_{t-1}|X_t)$ But now, we only know how to get $p(X_t|X_{t-1})$ which is the forward process. So how can we get the reverse process? How can we related conditional probability if we know one direction?

Yes! You are right!😎 We can use the Bayes' Theorem! But with a little trick by including $X_0$.

- So now, we want to get $q(X_{t-1}|X_t, X_0)$, and 
    - $q(X_{t-1}|X_t, X_0) = \frac{q(X_t|X_{t-1}, X_0)q(X_{t-1}|X_0)}{q(X_t|X_0)}$
    - Notice that all the terms in the right-hand side are known from the forward process
        - a little reminder: we got
            -  $X_t = \sqrt{a_t}X_{t-1} + \sqrt{1-a_t}Z_t$
            - $X_t = \sqrt{\bar a_t}X_0 + \sqrt{1 - \bar a_t}\bar Z$
    - $q(X_t|X_{t-1}, X_0) = \sqrt{a_t}X_{t-1} + \sqrt{1-a_t}Z_t$, which $ \sim N(\sqrt{a_t}X_{t-1}, 1-a_t)$
    - $q(X_{t-1}|X_0) = \sqrt{\bar a_{t-1}}X_0 + \sqrt{1 - \bar a_{t-1}}\bar Z$, which $ \sim N(\sqrt{\bar a_{t-1}}X_0, 1-\bar a_{t-1})$
    - $q(X_t|X_0) = \sqrt{\bar a_t}X_0 + \sqrt{1 - \bar a_t}\bar Z$, which $ \sim N(\sqrt{\bar a_t}X_0, 1-\bar a_t)$
- Now if we only consider $q(X_t|X_{t-1}, X_0)q(X_{t-1}|X_0)$
    - $q(X_t|X_{t-1}, X_0)q(X_{t-1}|X_0) = N(\sqrt{a_t}X_{t-1}, 1-a_t)N(\sqrt{\bar a_{t-1}}X_0, 1-\bar a_{t-1})$
    - = $\frac{1}{\sigma_1 \sqrt{2\pi}}e^{-\frac{1}{2} (\frac{x - \mu_1}{\sigma_1})^2} \cdot \frac{1}{\sigma_2 \sqrt{2\pi}}e^{-\frac{1}{2} (\frac{x - \mu_2}{\sigma_2})^2}$
    - = $\frac{1}{\sigma_1 \sigma_2 \sqrt{2\pi}}e^{-\frac{1}{2} (\frac{x - \mu_1}{\sigma_1})^2 - \frac{1}{2} (\frac{x - \mu_2}{\sigma_2})^2}$, where $\frac{1}{\sigma_1 \sigma_2 \sqrt{2\pi}}$ is constant
    - $\propto e^{-\frac{1}{2} (\frac{x - \mu_1}{\sigma_1})^2 - \frac{1}{2} (\frac{x - \mu_2}{\sigma_2})^2} = exp(-\frac{1}{2}(\frac{(X_t - \sqrt{a_t}X_{t-1})^2}{1-a_t} + \frac{(X_{t-1} - \sqrt{\bar a_{t-1}}X_{0})^2}{1- \bar a_{t-1}}))$
- Similarly when we include all three condtional probability:
    - $q(X_{t-1}|X_t, X_0) \propto exp(-\frac{1}{2}(\frac{(X_t - \sqrt{a_t}X_{t-1})^2}{1-a_t} + \frac{(X_{t-1} - \sqrt{\bar a_{t-1}}X_{0})^2}{1- \bar a_{t-1}} - \frac{(X_{t} - \sqrt{\bar a_t}X_{0})^2}{1- \bar a_{t}})) $
    - I know it's disheartening to see the formula above, but I will do the math for you😏.
    - $\propto exp(-\frac{1}{2}(\frac{X_t^2 - 2\sqrt{a_t}X_tX_{t-1} + a_tX_{t-1}^2}{1-a_t} + \frac{X_{t-1}^2 - 2\sqrt{\bar a_{t-1}}X_{t-1}X_0 + \bar a_{t-1}X_0^2}{1- \bar a_{t-1}} - \frac{X_{t}^2 - 2\sqrt{\bar a_t}X_tX_0 + \bar a_tX_0^2}{1- \bar a_{t}}))$
    - Since we only want to know the probability of $X_{t-1}$, we pair the terms with $X_{t-1}$ together
    - $= exp(-\frac{1}{2}[(\frac{a_t}{1-a_t} + \frac{1}{1-\bar a_{t-1}})X_{t-1}^2 + (\frac{2\sqrt{a_t}X_t}{1-a_t} - \frac{2\sqrt{\bar a_{t-1}}X_0}{1-\bar a_{t-1}})X_{t-1} + C(X_t, X_0)])$, where $C(X_t, X_0)$ is a term of $X_t$ and $X_0$. Since we don't care about $X_t$ and $X_0$, we treat it as a constant term.
    
- For any Gaussion distribution, we have:
    - $exp(-\frac{(x - \mu)^2}{2\sigma^2}) = exp(-\frac{1}{2}(\frac{1}{\sigma^2}x^2 - \frac{2\mu}{\sigma^2}x + \frac{\mu^2}{\sigma^2}))$

- By correspond the terms in the formula above, we can get:
    - $\frac{1}{\sigma^2} = (\frac{a_t}{1-a_t} + \frac{1}{1-\bar a_{t-1}})$, and $\beta_t = 1 - a_t$
    - $\frac{1}{\sigma^2} = \frac{a_t(1-\bar a_{t-1}) + \beta_t}{\beta_t(1-\bar a_{t-1})}$
    - $\sigma^2 = \frac{\beta_t(1-\bar a_{t-1})}{a_t(1-\bar a_{t-1}) + \beta_t}$

- Now we calculate our $\sigma^2$, the only thing left is the mean $\mu$ :
    - similarly, we have $\frac{2\mu}{\sigma^2} = (\frac{2\sqrt{a_t}X_t}{\beta_t} - \frac{2\sqrt{\bar a_{t-1}}X_0}{1-\bar a_{t-1}})$
    - => $\mu = \frac{\sigma^2}{2}(\frac{2\sqrt{a_t}X_t}{\beta_t} - \frac{2\sqrt{\bar a_{t-1}}X_0}{1-\bar a_{t-1}})$
    - => $\mu = \frac{\beta_t(1-\bar a_{t-1})}{a_t(1-\bar a_{t-1}) + \beta_t}(\frac{\sqrt{a_t}X_t(1-\bar a_{t-1}) - \beta_t \sqrt{\bar a_{t-1}}X_0}{\beta_t (1-\bar a_{t-1})})$
    - => $\mu = \frac{\sqrt{a_t}X_t(1-\bar a_{t-1}) - \beta_t \sqrt{\bar a_{t-1}}X_0}{a_t(1-\bar a_{t-1}) + \beta_t}$
    - => $\mu = \frac{\sqrt{a_t}X_t(1-\bar a_{t-1})}{a_t(1-\bar a_{t-1}) + \beta_t} - \frac{\beta_t \sqrt{\bar a_{t-1}}}{a_t(1-\bar a_{t-1}) + \beta_t}X_0$
- take a rest, we are almost there!😅
- Subsititute $\beta_t = 1 - a_t$ into the equation:
    - => $\mu = \frac{\sqrt{a_t}(1-\bar a_{t-1})}{a_t(1-\bar a_{t-1}) + 1 - a_t}X_t - \frac{\beta_t \sqrt{\bar a_{t-1}}}{a_t(1-\bar a_{t-1}) + 1 - a_t}X_0$
    - => $\mu = \frac{\sqrt{a_t}(1-\bar a_{t-1})}{1 - \bar a_{t-1}}X_t - \frac{\beta_t \sqrt{\bar a_{t-1}}}{1 - \bar a_{t-1}}X_0$

- But we don't have $X_0$ now, it's the result we want! So we need to get rid of $X_0$ in the equation. 
    - We can do this by using the formula we derived in the forward process: $X_t = \sqrt{\bar a_t}X_{0} + \sqrt{1-\bar a_t}Z_t$
    - => $\hat X_0 \sim X_0 = \frac{X_t - \sqrt{1-\bar a_t}Z_t}{\sqrt{\bar a_t}}$

- Now our equation would be:
    - => $\mu = \frac{\sqrt{a_t}(1-\bar a_{t-1})}{1 - \bar a_{t-1}}X_t - \frac{\beta_t \sqrt{\bar a_{t-1}}}{1 - \bar a_{t-1}} \cdot \frac{X_t - \sqrt{1-\bar a_t}Z_t}{\sqrt{\bar a_t}}$
    - => $\mu = \frac{1}{\sqrt{\bar a_t}}X_t - \frac{\beta_t}{\sqrt{\bar a_t}\sqrt{1-\bar a_t}} \cdot Z_t$
    - => $\mu = \frac{1}{\sqrt{\bar a_t}}(X_t - \frac{\beta_t}{\sqrt{1-\bar a_t}} \cdot Z_t)$

Finally, we got our mean $\mu$ and variance $\sigma^2$ for the reverse process. 🎉
We have:
- $\sigma^2 = \frac{\beta_t(1-\bar a_{t-1})}{a_t(1-\bar a_{t-1}) + \beta_t} = \frac{1- \bar a_{t-1}}{1- \bar a_{t}}\beta_t$
- $\mu = \frac{1}{\sqrt{\bar a_t}}(X_t - \frac{\beta_t}{\sqrt{1-\bar a_t}} \cdot Z_t)$

What the original paper said?

![](mu.png)

![](sigma.png)

Hurrah! We have the same results!🎉

# Model

Now, we don't have the $Z_t$ in our $\mu$, how should we get it? Train Deep Networks! 😎

What model should we use? I will save the answer for the implementation part! 😉

# Training

In training, we simply use the loss of $Z_t - \hat Z_t$ or MSE. Where $Z_t$ is from the forward pass, which is the noise added to the image for each timestep. And $\hat Z_t$ is from the reverse pass, which is the noise predicted by the model.

In short, the forward pass provides groud truth for training.

# Conclusion for the Math Part 📐

From now on NO MORE MATH! 😅

If you completely lost in the math part, don't worry! You are not gonna believe me, but it doesn't matter! (implementation-wise) 😂

The only two equaitions you need to remember are:
- $X_t = \sqrt{\bar a_t}X_0 + \sqrt{1 - \bar a_t}\bar Z$
- $\mu = \frac{1}{\sqrt{\bar a_t}}(X_t - \frac{\beta_t}{\sqrt{1-\bar a_t}} \cdot Z_t)$

# Pytorch Implementation

Here is the general structure of the model:

![](sd.png)

Which we have the following components:
- Encoder
- Decoder
- Clip Text Encoder
- Scheduler
- U-Net
And we will introduce them one by one in the implementation part.

# VAE Encoder

Instead of using the same diffusion structure from DDPM, the stable diffusion model is actually a lantent  diffusion model which it utilzied VAE encoder to encode the image into a latent space. The VAE encoder is a simple convolutional neural network that takes an image as input and outputs the mean and log variance of the latent space.

Basically, the structure of the VAE encoder is a comprised with a series of convolutional layers, residual blocks,one attention block, and lastly some convolutional layers. In general, the passing through each layer, the original input image is downsampled and the number of channels is increased. The attention block is used to capture the global context of the image. The output of the encoder is the mean and log variance of the latent space.

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import math

This is the resisual block structure used in encoder, which the structure is similar to the residual structure in the ResNet.

In [ ]:
class VAE_ResidualBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()
        self.groupnorm_1 = nn.GroupNorm(32, in_channels)
        self.conv_1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1)

        self.groupnorm_2 = nn.GroupNorm(32, out_channels)
        self.conv_2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1)
        
        # skip connection
        if in_channels != out_channels:
            self.residual_layer = nn.Conv2d(in_channels, out_channels, kernel_size = 1, padding=0)
        else:
            self.residual_layer = nn.Identity()
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x.shape = (batch_size, in_channels, height, width)
        residual = x
        x = self.groupnorm_1(x)
        x = F.silu(x)
        x = self.conv_1(x)
        x = self.groupnorm_2(x)
        x = F.silu(x)
        x = self.conv_2(x)

        return x + self.residual_layer(residual)

This is the selfAttention block structure used in encoder, which the structure is similar to the selfAttention structure in the Transformer.

In [ ]:
class SelfAttention(nn.Module):
    
    def __init__(self, n_heads: int, d_embed: int, in_proj_bias=True, out_proj_bias=True):
        super().__init__()
        
        self.in_proj = nn.Linear(d_embed, 3 * d_embed, bias=in_proj_bias) # Q, K, V
        self.out_proj = nn.Linear(d_embed, d_embed, bias=out_proj_bias) # W^O
        self.n_heads = n_heads
        self.d_head = d_embed // n_heads
    
    def forward(self, x: torch.Tensor, causal_mask=False) -> torch.Tensor:
        # x.shape = (batch_size, seq_len, dim)
        input_shape = x.shape
        batch_size, seq_len, d_embed = input_shape

        intermediate_shape = (batch_size, seq_len, self.n_heads, self.d_head)

        # (batch_size, seq_len, dim) -> (batch_size, seq_len, 3 * dim) -> 3 * (batch_size, seq_len, dim)
        q, k, v = self.in_proj(x).chunk(3, dim=-1)

        # (batch_size, seq_len, dim) -> (batch_size, seq_len, H, Dim/H) -> (batch_size, H, seq_len, Dim/H)
        q = q.view(intermediate_shape).transpose(1, 2)
        k = k.view(intermediate_shape).transpose(1, 2)
        v = v.view(intermediate_shape).transpose(1, 2)

        # (batch_size, H, seq_len, Dim/H) x (batch_size, H, Dim/H, seq_len) -> (batch_size, H, seq_len, seq_len)

        # Attention(Q, K, V) = softmax(QK^T / sqrt(d_k))V
        weight = torch.matmul(q, k.transpose(-2, -1))

        if causal_mask:
            mask = torch.ones_like(weight, dtype=torch.bool).triu(1)
            weight.masked_fill_(mask, -torch.inf)
        
        weight /= math.sqrt(self.d_head)
        weight = F.softmax(weight, dim=-1)

        # (batch_size, H, seq_len, seq_len) x (batch_size, H, seq_len, Dim/H) -> (batch_size, H, seq_len, Dim/H)
        output = torch.matmul(weight, v)
        # (batch_size, H, seq_len, Dim/H) -> (batch_size, seq_len, H, Dim/H)
        output = output.transpose(1, 2)

        # (batch_size, seq_len, H, Dim/H) -> (batch_size, seq_len, Dim)
        output = output.reshape(input_shape)

        # Multihead(Q, K, V) = Concat(head_1, ..., head_H)W^O
        output = self.out_proj(output)
        # (batch_size, seq_len, dim)
        return output


This is the attention block uses selfAttention block to capture the global context of the image.

In [ ]:
class VAE_AttentionBlock(nn.Module):
    def __init__(self, channels: int):
        super().__init__()
        self.groupnorm = nn.GroupNorm(32, channels)
        self.attention = SelfAttention(1, channels)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        residual = x
        x = self.groupnorm(x)
        batch_size, channels, height, width = x.shape

        # channels are the features
        # (batch_size, channels, height, width) -> (batch_size, channels, height * width)
        x = x.view(batch_size, channels, height * width)

        # (batch_size, channels, height * width) -> (batch_size, height * width, channels)
        x = x.transpose(-1, -2)

        # (batch_size, height * width, channels) -> (batch_size, height * width, channels)
        x = self.attention(x)

        # (batch_size, height * width, channels) -> (batch_size, channels, height * width)
        x = x.transpose(-1, -2)

        # (batch_size, channels, height * width) -> (batch_size, channels, height, width)
        x = x.view((batch_size, channels, height, width))

        # x = self.group_norm(x)
        x += residual

        return x

The actual implementation of the VAE encoder is shown below:

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

class VAE_Encoder(nn.Sequential):
    def __init__(self):
        super().__init__(
            # (batch_size, channels, height, width) -> (batch_size, 128, height, width)
            nn.Conv2d(3, 128, kernel_size = 3, padding = 1),

            VAE_ResidualBlock(128, 128),
            VAE_ResidualBlock(128, 128),

            # (batch_size, 128, height, width) -> (batch_size, 128, height/2, width/2)
            nn.Conv2d(128, 128, kernel_size = 3, stride = 2, padding = 0),

            # (batch_size, 128, height/2, width/2) -> (batch_size, 256, height/2, width/2)
            VAE_ResidualBlock(128, 256),
            VAE_ResidualBlock(256, 256),

            # (batch_size, 256, height/2, width/2) -> (batch_size, 256, height/4, width/4)
            nn.Conv2d(256, 256, kernel_size = 3, stride = 2, padding = 0),

            # (batch_size, 256, height/4, width/4) -> (batch_size, 512, height/4, width/4
            VAE_ResidualBlock(256, 512),
            VAE_ResidualBlock(512, 512),

            # (batch_size, 512, height/4, width/4) -> (batch_size, 512, height/8, width/8)
            nn.Conv2d(512, 512, kernel_size = 3, stride = 2, padding = 0),

            # (batch_size, 512, height/8, width/8)
            VAE_ResidualBlock(512, 512),
            VAE_ResidualBlock(512, 512),
            VAE_ResidualBlock(512, 512),

            # (batch_size, 512, height/8, width/8)
            VAE_AttentionBlock(512),
            
            VAE_ResidualBlock(512, 512),

            nn.GroupNorm(32, 512),
            nn.SiLU(),

            # (batch_size, 512, height/8, width/8) -> (batch_size, 8, height/8, width/8)
            # Bottleneck
            nn.Conv2d(512, 8, kernel_size = 3, padding = 1),

            # (batch_size, 8, height/8, width/8) -> (batch_size, 8, height/8, width/8)
            nn.Conv2d(8, 8, kernel_size = 1, padding=0),
        
        )

    def forward(self, x: torch.Tensor, noise = torch.Tensor) -> torch.Tensor:
    
        for layer in self:
            if getattr(layer, 'stride', None) == (2, 2):
                # Padding_left, padding_right, padding_top, padding_bottom
                x = F.pad(x, (0, 1, 0, 1))
            x = layer(x)

        # For VAE, want both the mean and the log variance
        # (batch_size, 8, height/8, width/8) -> two tensors with size (batch_size, 4, height/8, width/8)
        mean, log_variance = x.chunk(2, dim = 1)
        # Clamp the log variance to prevent the model from collapsing
        log_variance = torch.clamp(log_variance, -30, 20)

        # 1/2log(σ^2) = log(σ) -> σ = exp(0.5 * log_variance)
        std = (0.5 * log_variance).exp()

        # reparameterization trick
        if noise is None:
            noise = torch.randn_like(std)
        x = mean + std * noise

        # empirical scaling factor
        x *= 0.18215

        # return the samples from the latent space
        # (batch_size, 4, height/8, width/8)
        return x


# CLIP Text Encoder

TO BE CONTINUED...